In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("../data/train.csv")
validation = pd.read_csv("../data/valid.csv")
test = pd.read_csv("../data/test.csv")

In [3]:
train['label_2'].fillna(train['label_2'].mean(), inplace=True)
validation['label_2'].fillna(validation['label_2'].mean(), inplace=True)

In [4]:
train['label_2'] = train['label_2'].astype(int)
validation['label_2'] = validation['label_2'].astype(int)

In [5]:
from scipy import stats

def remove_outliers(df, threshold):
    # Copy the DataFrame
    df_copy = df.copy()
    
    # Remove the columns to exclude from the copy
    df_copy = df_copy.drop(["label_1", "label_2", "label_3", "label_4"], axis=1)
    
    # Calculate Z-scores only on remaining columns
    z_scores = stats.zscore(df_copy)
    abs_z_scores = np.abs(z_scores)
    
    # Identify outliers
    filtered_entries = (abs_z_scores < threshold).all(axis=1)
    
    # Apply the mask to the original DataFrame
    new_df = df[filtered_entries]
    
    return new_df

# Usage:
# exclude_cols is a list of column names to exclude
# train_df = remove_outliers(train_df, 3, exclude_cols)


In [6]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
def separate_dataset(df, label):
    X = df.drop(["label_1", "label_2", "label_3", "label_4"], axis=1)
    y = df[label]
    
    return X, y

def train_model(X_train, y_train, C=1.0, kernel='rbf', degree=3, gamma='scale'):
    model = svm.SVC(C=C, kernel=kernel, degree=degree, gamma=gamma, random_state=42)
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_train, X_test, y_train, y_test):
    print(f"Training score: {model.score(X_train, y_train)}")
    print(f"Testing score: {model.score(X_test, y_test)}")

def evaluate_model_detailed(model, X_test, y_test):
    y_pred = model.predict(X_test)

    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

In [8]:
test.head()

,ID,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768
0,1,0.124623,0.196628,0.257004,-0.156045,-0.054916,0.006071,-0.035149,-0.092019,-0.196302,...,-0.221466,0.140292,0.123622,-0.175572,-0.107030,-0.087621,-0.026501,0.139337,-0.083030,0.059507
1,2,0.109655,0.170158,0.227644,-0.127088,-0.044476,-0.046852,-0.090026,-0.061321,-0.227288,...,-0.204930,0.110203,0.085665,-0.286787,-0.113195,-0.057312,-0.055680,0.143939,-0.045760,0.106113
2,3,0.014854,0.030051,0.115092,-0.017179,0.002720,-0.011692,-0.078855,-0.042991,-0.096283,...,-0.032937,0.075821,0.030987,-0.149850,-0.003155,-0.010207,-0.001427,0.000934,-0.017069,0.048123
3,4,0.196893,0.113314,0.352175,-0.108499,-0.064472,-0.073239,-0.086402,0.008671,-0.342217,...,-0.255167,0.096579,0.069413,-0.215386,-0.075168,-0.035071,-0.023375,0.067768,-0.181530,0.174444
4,5,0.033004,0.013373,0.124001,-0.016143,0.010120,0.010635,-0.055789,-0.036282,-0.059422,...,-0.035814,0.093764,0.027321,-0.116009,0.010096,-0.042293,0.005347,0.007722,-0.007731,0.058799


In [9]:
X_train, y_train = separate_dataset(train, "label_3")
X_valid, y_valid = separate_dataset(validation, "label_3")
# X_test, y_test = separate_dataset(test, "label_2")
test_md = test.drop("ID", axis=1)

In [10]:
from sklearn.decomposition import PCA

# Create a PCA that will retain 99% of variance
pca = PCA(n_components=0.98, whiten=True)

# Fit PCA on the training set
X_train_pca = pca.fit_transform(X_train)

# Apply the transformation to the validation and test sets
X_valid_pca = pca.transform(X_valid)
X_test_pca = pca.transform(test_md)

print('Original number of features:', X_train.shape[1])
print('Reduced number of features:', X_train_pca.shape[1])


Original number of features: 768
Reduced number of features: 203


In [11]:
from sklearn.preprocessing import StandardScaler

def scale_features(X_train, X_test, X_val):
    # Initialize the scaler
    scaler = StandardScaler()
    
    # Save the column names
    columns = X_train.columns
    
    # Fit the scaler to the training data and transform
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Transform the test and validation data
    X_test_scaled = scaler.transform(X_test)
    X_val_scaled = scaler.transform(X_val)
    
    # Convert the scaled features into DataFrames
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=columns)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=columns)
    X_val_scaled_df = pd.DataFrame(X_val_scaled, columns=columns)
    
    return X_train_scaled_df, X_test_scaled_df, X_val_scaled_df


In [12]:
X_train_df = pd.DataFrame(X_train_pca)
X_test_df = pd.DataFrame(X_test_pca)
X_valid_df = pd.DataFrame(X_valid_pca)

In [13]:
X_train_scaled, X_test_scaled, X_valid_scaled = scale_features(X_train_df, X_test_df, X_valid_df)

In [14]:
model = train_model(X_train_scaled, y_train, degree=3, C=1.0, gamma="auto", kernel="rbf")

In [15]:
evaluate_model(model, X_train_scaled, X_valid_scaled, y_train, y_valid)
evaluate_model_detailed(model, X_valid_scaled, y_valid)

Training score: 0.9978611500701262
Testing score: 0.9933333333333333
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98       142
           1       0.99      1.00      1.00       608

    accuracy                           0.99       750
   macro avg       1.00      0.98      0.99       750
weighted avg       0.99      0.99      0.99       750

Confusion Matrix:
[[137   5]
 [  0 608]]


In [17]:
from sklearn.experimental import enable_halving_search_cv  # explicitly require this experimental feature
from sklearn.model_selection import HalvingRandomSearchCV  # noqa

param_dist = {
    'C': np.logspace(-4, 3, 8),  # expanded range
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    #'degree': [2, 3, 4, 5],  # added degree for 'poly' kernel
    'gamma': ['scale', 'auto'] + list(np.logspace(-4, 3, 8))  # expanded range
}

# Assuming X_train and y_train are your training data and labels
halving_random_search = HalvingRandomSearchCV(svm.SVC(random_state=42), param_dist, cv=3, verbose=10, n_jobs=-1, n_candidates=250)
halving_random_search.fit(X_train_scaled, y_train)

# Print the best parameters
print(halving_random_search.best_params_)

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 8
min_resources_: 12
max_resources_: 28520
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 250
n_resources: 12
Fitting 3 folds for each of 250 candidates, totalling 750 fits
----------
iter: 1
n_candidates: 84
n_resources: 36
Fitting 3 folds for each of 84 candidates, totalling 252 fits
----------
iter: 2
n_candidates: 28
n_resources: 108
Fitting 3 folds for each of 28 candidates, totalling 84 fits
----------
iter: 3
n_candidates: 10
n_resources: 324
Fitting 3 folds for each of 10 candidates, totalling 30 fits
----------
iter: 4
n_candidates: 4
n_resources: 972
Fitting 3 folds for each of 4 candidates, totalling 12 fits
----------
iter: 5
n_candidates: 2
n_resources: 2916
Fitting 3 folds for each of 2 candidates, totalling 6 fits
{'kernel': 'sigmoid', 'gamma': 0.0001, 'C': 100.0}


In [18]:
param_dist = {
    'C': np.logspace(-4, 3, 8),  # expanded range
    'kernel': [ 'poly' ],
    'degree': [2, 3, 4, 5],  # added degree for 'poly' kernel
    'gamma': ['scale', 'auto'] + list(np.logspace(-4, 3, 8))  # expanded range
}

# Assuming X_train and y_train are your training data and labels
halving_random_search = HalvingRandomSearchCV(svm.SVC(random_state=42), param_dist, cv=3, verbose=10, n_jobs=-1, n_candidates=250)
halving_random_search.fit(X_train_scaled, y_train)

# Print the best parameters
print(halving_random_search.best_params_)

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 8
min_resources_: 12
max_resources_: 28520
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 250
n_resources: 12
Fitting 3 folds for each of 250 candidates, totalling 750 fits
----------
iter: 1
n_candidates: 84
n_resources: 36
Fitting 3 folds for each of 84 candidates, totalling 252 fits
----------
iter: 2
n_candidates: 28
n_resources: 108
Fitting 3 folds for each of 28 candidates, totalling 84 fits
----------
iter: 3
n_candidates: 10
n_resources: 324
Fitting 3 folds for each of 10 candidates, totalling 30 fits
----------
iter: 4
n_candidates: 4
n_resources: 972
Fitting 3 folds for each of 4 candidates, totalling 12 fits
----------
iter: 5
n_candidates: 2
n_resources: 2916
Fitting 3 folds for each of 2 candidates, totalling 6 fits
{'kernel': 'poly', 'gamma': 'scale', 'degree': 4, 'C': 0.0001}


In [19]:
model_tuned = train_model(X_train_scaled, y_train, kernel="sigmoid", gamma=0.0001, degree=3, C=100.0)

In [20]:
evaluate_model(model_tuned, X_train_scaled, X_valid_scaled, y_train, y_valid)
evaluate_model_detailed(model_tuned, X_valid_scaled, y_valid)

Training score: 0.9908134642356241
Testing score: 0.9933333333333333
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       142
           1       1.00      1.00      1.00       608

    accuracy                           0.99       750
   macro avg       0.99      0.99      0.99       750
weighted avg       0.99      0.99      0.99       750

Confusion Matrix:
[[140   2]
 [  3 605]]


In [21]:
model_tuned1 = train_model(X_train_scaled, y_train, kernel="poly", gamma="scale", degree=4, C=0.0001)

In [22]:
evaluate_model(model_tuned1, X_train_scaled, X_valid_scaled, y_train, y_valid)
evaluate_model_detailed(model_tuned1, X_valid_scaled, y_valid)

Training score: 0.7995792426367462
Testing score: 0.8106666666666666
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       142
           1       0.81      1.00      0.90       608

    accuracy                           0.81       750
   macro avg       0.41      0.50      0.45       750
weighted avg       0.66      0.81      0.73       750

Confusion Matrix:
[[  0 142]
 [  0 608]]


d:\Python 3.10.11\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python 3.10.11\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python 3.10.11\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
